## Creamos Variable Retorno a la próxima ventana (12 meses futuro)

##### Esta variable requiere un paso extra y por ello se ha decidido separar en un notebook independiente. La variable retorno será importante en el cáculudo de la variable target. Si bien podemos coger el retorno a 12 meses pasado de la fila siguiente, algunas empresas salen del índice y por ello requiere consultar su precio a fecha de salida y calcular retornos así

In [1]:
import nasdaqdatalink as ndl
import pandas as pd
from ta import add_all_ta_features
import pandas as pd
import zipfile
import json

In [ ]:
#configurations
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [5]:
with open(r'..\config\ndl-config.json') as f:
    config = json.load(f)

api_key = config['API_KEY']
ndl.ApiConfig.api_key = api_key


In [2]:
merged = pd.read_csv(r'..\data\merged.csv', parse_dates=True)

In [5]:
merged.head()

,date,ticker,year_x,trade_date,close_reb_adj,vol_1y,max_1y,min_1y,sma_1y,sma_2y,...,currentratio,capex,ncfo,equity,de,year_y,rebalance_date,closeadj,ret_6m,ret_12m
0,2001-03-31,A,2001.0,2001-03-30,18.694,847576400.0,74.215,18.250,35.402956,38.759834,...,2.050,-702000000.0,838000000.0,5.265000e+09,0.600,2000.0,2001-03-31,18.694,-0.372073,-0.704513
1,2002-03-31,A,2002.0,2002-03-28,21.267,571608700.0,24.576,11.473,17.982242,26.952406,...,2.397,-594000000.0,-114000000.0,5.659000e+09,0.411,2001.0,2002-03-31,21.267,0.788195,0.137638
2,2003-03-31,A,2003.0,2003-03-31,7.999,677176600.0,22.222,6.600,11.631502,14.760365,...,2.238,-301000000.0,-585000000.0,4.627000e+09,0.773,2002.0,2003-03-31,7.999,0.006797,-0.623877
3,2004-03-31,A,2004.0,2004-03-31,19.241,653408900.0,22.806,8.352,15.060278,13.314752,...,2.040,-199000000.0,-144000000.0,2.824000e+09,1.230,2003.0,2004-03-31,19.241,0.430558,1.405426
4,2005-03-31,A,2005.0,2005-03-31,13.505,695043500.0,19.807,11.972,14.791570,14.926191,...,2.446,-96000000.0,663000000.0,3.569000e+09,0.977,2004.0,2005-03-31,13.505,0.029266,-0.298113


In [3]:

# Ordenar por ticker y date
merged = merged.sort_values(['ticker', 'date'])
# Calcular retorno a la próxima ventana para cada ticker
merged['target_12m_shift'] = merged.groupby('ticker')['ret_12m'].shift(-1)


#### 1. Importamos empresas que han dejado el índice, ya que su retorno no será el del año siguiente sino el calculado a fecha de salida. Dado que nuestro universo es el SP500 solo invertimos en empresas del índice

In [6]:
removed = ndl.get_table("SHARADAR/SP500",action=['removed'],date={'gte':'1998-01-01'}, paginate=True)
removed.df = removed[["date","ticker"]]
removed.to_csv(r"..\data\sp500_removed.csv", index=False)

C:\Users\Ignacio\AppData\Local\Temp\ipykernel_17692\1344144936.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  removed.df = removed[["date","ticker"]]


####  2. Identificamos las filas que necesitan tratamiento especial

In [7]:
# Vamos a identificar para cada ticker los tramos (periodos entre entrada y salida)
removed = removed.sort_values(['ticker', 'date'])

# Creamos un helper que para cada fila en merged nos diga si la fecha de salida es antes de la próxima ventana
import numpy as np

def get_next_exit_date(ticker, date):
    # Busca la PRÓXIMA fecha de salida de ese ticker después de la fecha actual
    salida = removed[(removed['ticker']==ticker) & (removed['date'] > date)]['date']
    if salida.empty:
        return np.nan
    else:
        return salida.iloc[0]

# Aplicamos la función a cada fila de merged
merged['next_exit_date'] = merged.apply(lambda row: get_next_exit_date(row['ticker'], row['date']), axis=1)


#### 3 Corregimos el target para los casos donde la salida ocurre antes de la próxima ventana

In [11]:
# Importar csv con precios ajustados
prices = pd.read_csv(
    r"../largefiles/prices.csv.gz",
    compression="gzip",
    parse_dates=["date"]
)[["ticker", "date", "closeadj"]]

In [12]:
import pandas as pd

# Convierte todas las fechas relevantes a datetime
for df in [merged, removed, prices]:
    for col in ['date', 'next_exit_date']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])


In [13]:
# Helper para buscar el precio más cercano a la fecha de salida
from datetime import timedelta

def get_exit_price_near(ticker, exit_date, margin_days=15):
    # Limita al margen de ±15 días
    date_min = pd.to_datetime(exit_date) - timedelta(days=margin_days)
    date_max = pd.to_datetime(exit_date) + timedelta(days=margin_days)
    price = prices[
        (prices['ticker'] == ticker) & 
        (prices['date'] >= date_min) & 
        (prices['date'] <= date_max)
    ].copy()
    if price.empty:
        return np.nan
    # Elige la fecha más cercana a exit_date (absoluta)
    price['date_diff'] = (price['date'] - pd.to_datetime(exit_date)).abs()
    nearest_row = price.sort_values('date_diff').iloc[0]
    return nearest_row['closeadj']


# Ahora creamos la columna para el retorno a salida cuando corresponda
def calc_exit_return(row):
    # Caso 1: No hay salida registrada o salida después de la próxima ventana -> usar shift normal
    if pd.isna(row['next_exit_date']):
        return row['target_12m_shift']
    # Si la salida es antes de la próxima ventana, calcular retorno a la salida
    next_shift_date = merged[
        (merged['ticker'] == row['ticker']) & 
        (merged['date'] > row['date'])
    ]['date'].min()
    if pd.isna(next_shift_date) or row['next_exit_date'] < next_shift_date:
        exit_price = get_exit_price_near(row['ticker'], row['next_exit_date'])
        if pd.isna(exit_price):
            return np.nan
        return (exit_price - row['closeadj']) / row['closeadj']
    else:
        # Si la próxima ventana ocurre antes de la salida, usar shift normal
        return row['target_12m_shift']

merged['target_12m_final'] = merged.apply(calc_exit_return, axis=1)


In [14]:
merged.to_csv('../data/merged_with_returns.csv', index=False)